In [1]:
from larlib import *
from math import floor
import csv
import os

Evaluating fenvs.py..
...fenvs.py imported in 0.00606 seconds


# Reading data
As mentioned in workshop_05 I will assign 3d-points instead of getting data from the filesystem. This will prevent problems regarding paths etc. this approach may come in handy for fine tune points ;)

In [1]:
def larModel(path2lines):
    """
    larModel("mypath")
    :param path2file: path to the .lines file
    :return: tuple. A graph model representing Vertices and Edges.
             Edges are lists of indices to Vertices.
    """
    with open(path2lines, "rb") as file:
        lines = csv.reader(file, delimiter=",")

        mapping = {}
        v = []
        ev = []
        index = 0
        
        # indexing of the vertices and approximation
        for entry in lines:
            x1 = math.floor(float(entry[0])*100)/100
            y1 = math.floor(float(entry[1])*100)/100
            x2 = math.floor(float(entry[2])*100)/100
            y2 = math.floor(float(entry[3])*100)/100

            if not(mapping.has_key((x1,y1))):
                index += 1
                mapping[(x1,y1)] = index
                v.append([x1,y1])

            if not(mapping.has_key((x2,y2))):
                index += 1
                mapping[(x2,y2)] = index
                v.append([x2,y2])

            ev.append([mapping[(x1,y1)], mapping[(x2,y2)]])

    return v,ev

# Work pipeline
- Divide et impera: I will start off with splitting the structure.
  For every part of Villa Badoer a .csv and a .lines will be produced (https://inkscape.org)
- Given a .lines file for a certain part of the structure, produce a graph (V,EV) as lar model
- Given a part of the villa, draw its plant
- Given a plant use cartesian product to build walls etc.
- Figure out a way to use symmetries